In [1]:
import pytrends
import pandas as pd
import os
import pycountry
import trends_helpers
import seaborn as sns
import matplotlib.pyplot as plt

Looking for four search terms ('passport', 'consulate', 'migration', 'visa', 'inflation') in different countries, with different time frames:
+ Afghanistan: AF
+ Syria: SY
+ Myanmar: MM
+ Democratic Republic of Congo: CD
+ Uruguay: UY
+ Argentina: AR
+ Ethiopia: ET
+ Eritrea: ER
+ China: CN
+ India: IN
Just to do some hypothesis testing, we'll do it in English.

In [2]:
countries = ["AF", "SY", "MM", "CD", "VE", "CO", "UY", "AR", "ET", "ER", "CN", "IN"]
keywords_list = ["passport", "consulate", "migration", "visa", "inflation"]
timeframes= ['all', 
            "2005-01-01 2022-01-01",
            "2006-01-01 2022-01-01",
            "2007-01-01 2022-01-01",
            "2008-01-01 2022-01-01",
            "2004-01-01 2007-12-31"]

### Some initial tests

#### Correlations between trends, considering same periods.

+ 2008-2021

In [3]:
results = pd.read_csv("testing gtrends_sample.csv").drop(columns=["Unnamed: 0"])
results.head()

,date,trends_index,keyword,region,timeframe
0,2004-01-01,100,passport,AF,all
1,2004-02-01,0,passport,AF,all
2,2004-03-01,0,passport,AF,all
3,2004-04-01,0,passport,AF,all
4,2004-05-01,6,passport,AF,all


In [4]:
results_2008 = results[(results["date"]>"2007-12-31") & (results["date"]<"2022-01-01")]

wide_data= pd.DataFrame()
for c in countries:
    for k in keywords_list:
        filtered = results_2008[(results_2008["region"]==c) & (results_2008["keyword"]==k)]
        wide = filtered.pivot(index='date', columns='timeframe', values='trends_index')
        wide["country"]=c
        wide["keyword"]=k
        wide_data = pd.concat([wide_data, wide])


In [9]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html
import seaborn as sns
import matplotlib.pyplot as plt
def corplots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    # Filter the data based on the selected country and keyword
    country_data = wide_data[(wide_data["country"] == country) & (wide_data["keyword"] == keyword)]
    corr_matrix = country_data[['all', '2005-01-01 2022-01-01', '2006-01-01 2022-01-01', '2007-01-01 2022-01-01', '2008-01-01 2022-01-01']].corr()

    # Create the correlation plot using seaborn
    fig = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.4f')
    fig.set_title(str(keyword) + ' in ' + str(country))

    # Show the plot
    plt.show()

# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(corplots, country=country_dropdown, keyword=keyword_dropdown)


interactive(children=(Dropdown(description='Country:', options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', 'UY', 'AR'…

<function __main__.corplots(country, keyword)>

Analyzing and comparing trends:

In [6]:
adjusted = trends_helpers.adjust_trend(df=wide_data, columns_to_adjust=list(wide_data.columns[0:5,]))
adjusted = adjusted.reset_index()
adjusted["date"] = pd.to_datetime(adjusted['date'])
adjusted.head()

timeframe,date,2005-01-01 2022-01-01,2006-01-01 2022-01-01,2007-01-01 2022-01-01,2008-01-01 2022-01-01,all,country,keyword,2005-01-01 2022-01-01_adjusted,2006-01-01 2022-01-01_adjusted,2007-01-01 2022-01-01_adjusted,2008-01-01 2022-01-01_adjusted,all_adjusted
0,2008-01-01,20.0,22.0,35.0,51.0,1.0,AF,passport,50.0,51.162791,51.470588,51.0,100.0
1,2008-02-01,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.000000,0.000000,0.0,0.0
2,2008-03-01,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.000000,0.000000,0.0,0.0
3,2008-04-01,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.000000,0.000000,0.0,0.0
4,2008-05-01,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.000000,0.000000,0.0,0.0


In [7]:
list_adjusted_cols= ['2005-01-01 2022-01-01_adjusted',
       '2006-01-01 2022-01-01_adjusted', '2007-01-01 2022-01-01_adjusted',
       '2008-01-01 2022-01-01_adjusted', 'all_adjusted']

In [8]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html
# Create dropdown widgets for selecting the country and keyword
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
keyword_dropdown = widgets.Dropdown(options=keywords_list, description='Keyword:')

def update_plots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    country_data = adjusted[(adjusted["country"] == country) & (adjusted["keyword"] == keyword)]
    
    fig = px.line(country_data, x="date", y=list_adjusted_cols, title=f"Searches for {keyword} in {country}")
    fig.show()
# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(update_plots, country=country_dropdown, keyword=keyword_dropdown)


interactive(children=(Dropdown(description='Country:', options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', 'UY', 'AR'…

<function __main__.update_plots(country, keyword)>

+ 2007-2021

In [36]:
results_2007 = results[(results["date"]>"2006-12-31") & (results["date"]<"2022-01-01")]

wide_data= pd.DataFrame()
for c in countries:
    for k in keywords_list:
        filtered = results_2007[(results_2007["region"]==c) & (results_2007["keyword"]==k)]
        wide = filtered.pivot(index='date', columns='timeframe', values='trends_index')
        wide["country"]=c
        wide["keyword"]=k
        wide_data = pd.concat([wide_data, wide])
        wide_data = wide_data.fillna(0)

In [37]:
wide_data.columns

Index(['2004-01-01 2007-12-31', '2005-01-01 2022-01-01',
       '2006-01-01 2022-01-01', '2007-01-01 2022-01-01',
       '2008-01-01 2022-01-01', 'all', 'country', 'keyword'],
      dtype='object', name='timeframe')

In [38]:
adjusted = trends_helpers.adjust_trend(df=wide_data, columns_to_adjust=list(wide_data.columns[1:6,]))
adjusted = adjusted.reset_index()
adjusted["date"] = pd.to_datetime(adjusted['date'])
adjusted.head()

timeframe,date,2004-01-01 2007-12-31,2005-01-01 2022-01-01,2006-01-01 2022-01-01,2007-01-01 2022-01-01,2008-01-01 2022-01-01,all,country,keyword,2005-01-01 2022-01-01_adjusted,2006-01-01 2022-01-01_adjusted,2007-01-01 2022-01-01_adjusted,2008-01-01 2022-01-01_adjusted,all_adjusted
0,2007-01-01,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
1,2007-01-07,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
2,2007-01-14,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
3,2007-01-21,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
4,2007-01-28,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0


In [39]:
adjusted.columns

Index(['date', '2004-01-01 2007-12-31', '2005-01-01 2022-01-01',
       '2006-01-01 2022-01-01', '2007-01-01 2022-01-01',
       '2008-01-01 2022-01-01', 'all', 'country', 'keyword',
       '2005-01-01 2022-01-01_adjusted', '2006-01-01 2022-01-01_adjusted',
       '2007-01-01 2022-01-01_adjusted', '2008-01-01 2022-01-01_adjusted',
       'all_adjusted'],
      dtype='object', name='timeframe')

In [40]:
list_adjusted_cols= ['2005-01-01 2022-01-01_adjusted', '2006-01-01 2022-01-01_adjusted',
       '2007-01-01 2022-01-01_adjusted', '2008-01-01 2022-01-01_adjusted',
       'all_adjusted']

In [41]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html
# Create dropdown widgets for selecting the country and keyword
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
keyword_dropdown = widgets.Dropdown(options=keywords_list, description='Keyword:')

def update_plots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    country_data = adjusted[(adjusted["country"] == country) & (adjusted["keyword"] == keyword)]
    
    fig = px.line(country_data, x="date", y=list_adjusted_cols, title=f"Searches for {keyword} in {country}")
    fig.show()
# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(update_plots, country=country_dropdown, keyword=keyword_dropdown)

interactive(children=(Dropdown(description='Country:', options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', 'UY', 'AR'…

<function __main__.update_plots(country, keyword)>

+ 2006-2021

In [42]:
results_2006 = results[(results["date"]>"2005-12-31") & (results["date"]<"2022-01-01")]

wide_data= pd.DataFrame()
for c in countries:
    for k in keywords_list:
        filtered = results_2006[(results_2006["region"]==c) & (results_2006["keyword"]==k)]
        wide = filtered.pivot(index='date', columns='timeframe', values='trends_index')
        wide["country"]=c
        wide["keyword"]=k
        wide_data = pd.concat([wide_data, wide])
        wide_data = wide_data.fillna(0)

In [43]:
adjusted = trends_helpers.adjust_trend(df=wide_data, columns_to_adjust=list(wide_data.columns[1:6,]))
adjusted = adjusted.reset_index()
adjusted["date"] = pd.to_datetime(adjusted['date'])
adjusted.head()

timeframe,date,2004-01-01 2007-12-31,2005-01-01 2022-01-01,2006-01-01 2022-01-01,2007-01-01 2022-01-01,2008-01-01 2022-01-01,all,country,keyword,2005-01-01 2022-01-01_adjusted,2006-01-01 2022-01-01_adjusted,2007-01-01 2022-01-01_adjusted,2008-01-01 2022-01-01_adjusted,all_adjusted
0,2006-01-01,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
1,2006-01-08,12.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
2,2006-01-15,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
3,2006-01-22,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0
4,2006-01-29,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.0,0.0,0.0,0.0,0.0


In [44]:
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.embed import embed_minimal_html
# Create dropdown widgets for selecting the country and keyword
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
keyword_dropdown = widgets.Dropdown(options=keywords_list, description='Keyword:')

def update_plots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    country_data = adjusted[(adjusted["country"] == country) & (adjusted["keyword"] == keyword)]
    
    fig = px.line(country_data, x="date", y=list_adjusted_cols, title=f"Searches for {keyword} in {country}")
    fig.show()
# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(update_plots, country=country_dropdown, keyword=keyword_dropdown)

interactive(children=(Dropdown(description='Country:', options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', 'UY', 'AR'…

<function __main__.update_plots(country, keyword)>

+ Testing by aggregating monthly trends

In [67]:
results_2006 = results[(results["date"]>"2005-12-31") & (results["date"]<"2022-01-01")]
results_2006.set_index('date', inplace=True)
results_2006.index = pd.DatetimeIndex(results_2006.index)

# Group by 'Country', 'Keyword', and 'Timeframe' columns and resample by month, then mean the values
monthly_aggregated = results_2006.groupby(['region', 'keyword', 'timeframe']).resample('M').mean().reset_index()

monthly_aggregated

,region,keyword,timeframe,date,trends_index
0,AF,consulate,2004-01-01 2007-12-31,2006-01-31,0.00
1,AF,consulate,2004-01-01 2007-12-31,2006-02-28,1.25
2,AF,consulate,2004-01-01 2007-12-31,2006-03-31,0.00
3,AF,consulate,2004-01-01 2007-12-31,2006-04-30,2.00
4,AF,consulate,2004-01-01 2007-12-31,2006-05-31,0.00
...,...,...,...,...,...
53083,VE,visa,all,2021-08-31,33.00
53084,VE,visa,all,2021-09-30,30.00
53085,VE,visa,all,2021-10-31,35.00
53086,VE,visa,all,2021-11-30,29.00


In [69]:
wide_data= pd.DataFrame()
for c in countries:
    for k in keywords_list:
        filtered = monthly_aggregated[(monthly_aggregated["region"]==c) & (monthly_aggregated["keyword"]==k)]
        wide = filtered.pivot(index='date', columns='timeframe', values='trends_index')
        wide["country"]=c
        wide["keyword"]=k
        wide_data = pd.concat([wide_data, wide])
        wide_data = wide_data.fillna(0)

In [70]:
wide_data

timeframe,2004-01-01 2007-12-31,2005-01-01 2022-01-01,2006-01-01 2022-01-01,2007-01-01 2022-01-01,2008-01-01 2022-01-01,all,country,keyword
date,,,,,,,,
2006-01-31,2.4,0.0,0.0,0.0,0.0,0.0,AF,passport
2006-02-28,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport
2006-03-31,0.0,77.0,84.0,0.0,0.0,2.0,AF,passport
2006-04-30,0.6,92.0,100.0,0.0,0.0,2.0,AF,passport
2006-05-31,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport
...,...,...,...,...,...,...,...,...
2021-08-31,0.0,9.0,9.0,9.0,9.0,9.0,IN,inflation
2021-09-30,0.0,8.0,8.0,8.0,8.0,8.0,IN,inflation
2021-10-31,0.0,8.0,8.0,8.0,8.0,8.0,IN,inflation


In [71]:
wide_data.columns

Index(['2004-01-01 2007-12-31', '2005-01-01 2022-01-01',
       '2006-01-01 2022-01-01', '2007-01-01 2022-01-01',
       '2008-01-01 2022-01-01', 'all', 'country', 'keyword'],
      dtype='object', name='timeframe')

In [72]:
adjusted = trends_helpers.adjust_trend(df=wide_data, columns_to_adjust=list(wide_data.columns[1:6,]))
adjusted = adjusted.reset_index()
adjusted["date"] = pd.to_datetime(adjusted['date'])
adjusted.head()

timeframe,date,2004-01-01 2007-12-31,2005-01-01 2022-01-01,2006-01-01 2022-01-01,2007-01-01 2022-01-01,2008-01-01 2022-01-01,all,country,keyword,2005-01-01 2022-01-01_adjusted,2006-01-01 2022-01-01_adjusted,2007-01-01 2022-01-01_adjusted,2008-01-01 2022-01-01_adjusted,all_adjusted
0,2006-01-31,2.4,0.0,0.0,0.0,0.0,0.0,AF,passport,0.000000,0.0,0.0,0.0,0.0
1,2006-02-28,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.000000,0.0,0.0,0.0,0.0
2,2006-03-31,0.0,77.0,84.0,0.0,0.0,2.0,AF,passport,83.695652,84.0,0.0,0.0,100.0
3,2006-04-30,0.6,92.0,100.0,0.0,0.0,2.0,AF,passport,100.000000,100.0,0.0,0.0,100.0
4,2006-05-31,0.0,0.0,0.0,0.0,0.0,0.0,AF,passport,0.000000,0.0,0.0,0.0,0.0


In [73]:
# Create dropdown widgets for selecting the country and keyword
country_dropdown = widgets.Dropdown(options=countries, description='Country:')
keyword_dropdown = widgets.Dropdown(options=keywords_list, description='Keyword:')

def update_plots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    country_data = adjusted[(adjusted["country"] == country) & (adjusted["keyword"] == keyword)]
    
    fig = px.line(country_data, x="date", y=list_adjusted_cols, title=f"Searches for {keyword} in {country}")
    fig.show()
# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(update_plots, country=country_dropdown, keyword=keyword_dropdown)

interactive(children=(Dropdown(description='Country:', options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', 'UY', 'AR'…

<function __main__.update_plots(country, keyword)>

In [74]:
def corplots(country, keyword):
    # Clear the output area
    clear_output(wait=True)
    
    # Filter the data based on the selected country and keyword
    country_data = wide_data[(wide_data["country"] == country) & (wide_data["keyword"] == keyword)]
    corr_matrix = country_data[['all', '2005-01-01 2022-01-01', '2006-01-01 2022-01-01', '2007-01-01 2022-01-01', '2008-01-01 2022-01-01']].corr()

    # Create the correlation plot using seaborn
    fig = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.4f')
    fig.set_title(str(keyword) + ' in ' + str(country))

    # Show the plot
    plt.show()

# Register the update_plots function as the callback for the dropdown widgets
widgets.interact(corplots, country=country_dropdown, keyword=keyword_dropdown)

interactive(children=(Dropdown(description='Country:', index=6, options=('AF', 'SY', 'MM', 'CD', 'VE', 'CO', '…

<function __main__.corplots(country, keyword)>